Neuron Lang is a python based DSL for naming neurons. Neurons are modelled as collections of phenotypes with semantics backed by Web Ontology Language (OWL2) classes. Neuron Lang provides tools for mapping to and from collections of local names for phenotypes by using ontology identifiers as the common language underlying all local naming. These tools also let us automatically generate names for neurons in a regular and consistent way using a set of rules operating on the neurons' constitutent phenotypes. Neuron Lang can export to python or to any serialziation supported by rdflib, however [deterministic turtle](https://github.com/tgbugs/pyontutils/blob/master/docs/ttlser.md) (ttl) is prefered.

This notebook has examples of how to use Neuron Lang to:
* Define neurons and phenotypes.
* Export all defined neurons.
* Use `%scig` magic to search for existing ontology identifiers
* Use `LocalNameManager` to create abbreviations for phenotypes.
* Bind local names in the current python namespace using `with` or `setLocalNames`.
* Creat a phenotype context in which to define neurons using `with` or `setLocalContext`.

Please see [the documentation](https://github.com/tgbugs/pyontutils/blob/master/docs/neurons_notebook.md) in order to set up a working
environment for this notebook.

In [ ]:
# basic setup for any file defining neurons

from pyontutils.neuron_lang import *
from pyontutils import phenotype_namespaces
config(out_graph_path='neuron_lang_example_neurons.ttl')

# WARNING only call config once at the start of your program
# if you call it again it will reset the state and can be very confusing
# see neuron_lang.config and neurons.graphBase.configGraphIO for details

In [ ]:
# neurons

myFirstNeuron = Neuron(Phenotype('NCBITaxon:10090'),
                       Phenotype('UBERON:0000955'))

# Neuron instances are build out of Phenotype instances.
# Phenotypes are object-predicate pairs that take curied
# string representations (or uris) as arguments.
# NOTE: label is cosmetic and will be overwritten by rdfs:label

myPhenotype = Phenotype('NCBITaxon:9685',  # object
                        'ilx:hasInstanceInSpecies',  # predicate (optional)
                        label='Cat')  # label for human readability

# print and repr produce different results

print(myFirstNeuron)
print(repr(myFirstNeuron))

# view the turtle (ttl) serialization of all neurons

turtle = graphBase.ttl()
print('---turtle export---', turtle, sep='\n')

# view the python serialization of all neurons

python = graphBase.python()
print('---python export---', python, sep='\n')

# write turtle and python to file

graphBase.write()
graphBase.write_python()

# get a list of all defined neurons

graphBase.neurons()

# NOTE you can also use Neurons.neurons() & co. instead
# the functionality is the same as calling graphBase.neurons()
# but graphBase is what holds the global state for converting
# classes to turtle and managing serialization

In [ ]:
# scig

# When creating neurons we want to be able to find relevant identifiers
# quickly while working. There is a cli utility called scig that can be
# used as a cell magic %scig to search a SciGraph instance for terms.

%scig --help
print('---Terms---')
# use -t to limit the number of results
%scig -t 1 t hippocampus
# you can escape spaces with \
%scig t macaca\ mulatta  
print('---Search---')
# quotes also allow search with spaces
%scig -t 1 s 'nucleus basalis of meynert'
# without quotes scig will search multiple terms at once
%scig -t 1 t cat mouse

# programatic access to a SciGraph instance works as follows
from pyontutils.scigraph_client import Graph, Vocabulary

sgg = Graph()
sgv = Vocabulary()

terms = sgv.findByTerm('neocortex')
nodes_edges = sgg.getNeighbors('UBERON:0000955', 
                               relationshipType='BFO:0000050',  # part of
                               direction='INCOMING')
print(terms[0]['synonyms'])
print(*(e['sub'] for e in nodes_edges['edges']))

In [ ]:
# namespaces - context managers

# we can be more concise by creating a namespace for our phenotype names
# normally these are defined in another file so they can be reused

from inspect import stack  # welcome to python metaprogramming >_<
from pyontutils.neurons import LocalNameManager, addLN, addLNT

class myPhenotypeNames(LocalNameManager):  # see neurons.LocalNameManager
    Mouse = Phenotype('NCBITaxon:10090', 'ilx:hasInstanceInSpecies')
    Rat = Phenotype('NCBITaxon:10116', 'ilx:hasInstanceInSpecies')
    brain = Phenotype('UBERON:0000955', 'ilx:hasSomaLocatedIn')
    PV = Phenotype('PR:000013502', 'ilx:hasExpressionPhenotype')

# with a context manager we can use a namespace to create neurons
# more concisely and more importantly to repr them more concisely

with myPhenotypeNames():
    n = Neuron(Rat, brain, PV)
    print('inside', repr(n))
    
    # printing is unaffected so the fully expanded form is always
    # accessible (__str__ vs __repr__)
    
    print(n)
    
    # we can also repr a neuron defined elsewhere using our own names
    
    print('from outside', repr(myFirstNeuron))
    
    
# outside the context manager our concise repr is gone

print('outside', repr(n))

# we will now get a NameError of we try to use bare words

try: Neuron(Rat)
except NameError: print('Rat fails as expected.')
    
# for a full explication of phenotype namespaces see neuron_example.py

In [ ]:
# namespaces 2 - global modification

# there are already many namespaces defined in phenotype_namespaces.py
print(phenotype_namespaces.__all__)

# setLocalNames adds any names from a namespace to the current namespace
setLocalNames(phenotype_namespaces.Species)

# we can load additional names but will get a ValueError on a conflict
setLocalNames(phenotype_namespaces.Regions, phenotype_namespaces.Layers)
try:
    setLocalNames(phenotype_namespaces.Test)
except ValueError as e:
    print(e)

# we can extend namespaces as well (again, best in a separate file)
# as long as the local names match we can combine entries
class MoreSpecies(phenotype_namespaces.Species, myPhenotypeNames):
    addLN('Cat', myPhenotype)
    addLNT('ACh', 'CHEBI:15355', 'ilx:hasExpressionPhenotype')
    # using addLN/addLNT allows us to immediately use those names
    addLN('AChMinus', NegPhenotype(ACh))
    
with MoreSpecies():
    can = Neuron(Cat, ACh, L2)
    cant = Neuron(Cat, AChMinus, L3)
    print(can, cant, sep='\n')

# we can also refer to phenotypes in a namespace directly
n = Neuron(Mouse, MoreSpecies.ACh)
print(n)

# getLocalNames can be used to inspect the current set of defined names
print(sorted(getLocalNames().keys()))

# clear the local names by calling setLocalNames with no arguments
setLocalNames()

# no more short names ;_;
try: Neuron(Mouse, PV)
except NameError: print('Neuron(Mouse, PV) fails as expected')

# for the rest of these examples we will use the BBP namespace
setLocalNames(phenotype_namespaces.BBP)

# define some new neurons using our local names
Neuron(Mouse, L23, CCK, NPY)
Neuron(Mouse, brain, L3, PV)
Neuron(PV, DA)

In [ ]:
# context - context managers

# we often want to create many neurons in the same contex
# the easiest way to do this is to use a instance of a neuron
# as the input to a context manager
with Neuron(Rat, CA1):
    n1 = Neuron(CCK)
    n2 = Neuron(NPY)
    n3 = Neuron(PC)

# neurons always retain the context they were created in
print('example 1', *map(repr, (n1, n2, n3)), sep='\n')

# you cannot change a neuron's context but you can see its original context
print('', 'example 2', n3.context, sep='\n')
try:
    n3.context = Neuron(Mouse, CA2)
except TypeError as e:
    print(e)

# you can also use with as syntax when creating a context
with Neuron(Mouse) as n4:
    n5 = Neuron(CCK)

print('', 'example 3', *map(repr, (n4, n5)), sep='\n')
    
# context does not nest for neurons defined outside a with (this may change)
with n3:
    n6 = Neuron(VIP)
    with n5:  # does not nest
        n7 = Neuron(SOM)
    with Neuron(Mouse) as n8:  # nests
        n9 = Neuron(SOM)
    n10 = Neuron(SOM)

print('', 'example 4', *map(repr, (n3, n6, n5, n7, n8, n9, n10)), sep='\n')

# Side note
# From these examples we see that Neuron currently does not check
# if the phenotypes it is given are disjoint (e.g. Rat Mouse).
# For the time being that should be handled in a second step by
# the ontology reasoner. There are some rdflib Fact++ bindings that
# might be relevant in the future.

In [ ]:
# context 2 - global modification

# like namespaces you can also set a persistent local context
setLocalContext(Neuron(CCK, NPY, SOM, DA, CA1, SPy))
print(repr(Neuron(brain)))

# unlike namespaces contexts are not addative (yet)
setLocalContext(Neuron(Rat, S1, L4))
print(repr(Neuron(brain)))

# like namespaces call getLocalContext to see the current context
print(*(p.pShortName for p in getLocalContext()))

# like namespaces calling setLocalContext without arguments clears context
setLocalContext()
print(repr(Neuron(brain)))

In [ ]:
# context 3 - the old way
setLocalContext()  # clear existing context

context = (Rat, S1)
ca1_context = (Rat, CA1)

def NeuronC(*args, **kwargs):
    return Neuron(*args, *context, **kwargs)

def NeuronH(*args, **kwargs):
    return Neuron(*args, *ca1_context, **kwargs)

neurons = {
    'HBP_CELL:0000013': NeuronC(CCK),
    'HBP_CELL:0000016': NeuronC(PV),
    'HBP_CELL:0000018': NeuronC(PC),
    'HBP_CELL:0000135': NeuronH(SLM, PPA),
    'HBP_CELL:0000136': NeuronH(SO, BP),
    'HBP_CELL:0000137': NeuronH(SPy, BS),
    'HBP_CELL:0000148': Neuron(Rat, STRI, MSN, D1),
    'HBP_CELL:0000149': Neuron(Rat, CA3, PC),
        }
neurons['HBP_CELL:0000013']